In [1]:
import os
import math
import numpy as np
import pandas as pd
import pywt

In [2]:
def wavefilter(data):
    """
    This function requires that the NumPy and PyWavelet packages are installed. They are freely available at:
    Filter a multi-channel signal using wavelet filtering.
    data     - n x m array, n=number of channels in the signal,
                m=number of samples in the signal
    maxlevel - the level of decomposition to perform on the data. This integer
                implicitly defines the cutoff frequency of the filter.
                Specifically, cutoff frequency = samplingrate/(2^(maxlevel+1))
    """
    # We will use the Daubechies(6) wavelet
    daubechies_num = 6
    wname = "db" + str(daubechies_num)
    
    datalength = data.shape[0]
    #datalength = len(data)
    max_level = pywt.dwt_max_level(datalength, wname)
    #print('maximun level is: %s' % max_level)
    # Initialize the container for the filtered data
    # Decompose the signal
    temp_c = pywt.wavedec(data, wname, mode='smooth', level=max_level)
    temp_c_array = np.array(temp_c, dtype=object)
    # coeff[0] is approximation coeffs, coeffs[1] is nth level detail coeff, coeff[-1] is first level detail coeffs
    coeffs = pywt.wavedec(data, wname, mode='smooth', level=max_level)
    # hard thresholding based on universal threshold
    # detail_coeffs_n = coeffs[-1]  # note that the finest scale coefficients should be used here
    # temp_mad = stats.median_absolute_deviation(detail_coeffs_n)
    # universal_threshold = stats.median_absolute_deviation(detail_coeffs_n) * math.sqrt(2*math.log(datalength))
    # for j in range(max_level):
    #     before_threshold = coeffs[-j-1]
    #     # after_threshold = pywt.threshold(before_threshold, universal_threshold, mode='hard')
    #     after_threshold = pywt.threshold(before_threshold, universal_threshold, mode='soft')
    #     coeffs[-j-1] = after_threshold
    # directly set all details to 0, this is probably the best result we can get here
    for j in range(max_level):
        coeffs[-j - 1] = np.zeros_like(coeffs[-j - 1])
    # Reconstruct the signal and save it
    filter_data = pywt.waverec(coeffs, wname, mode='smooth')
    fdata = filter_data[0:datalength]
    return fdata  # Otherwise, give back the 2D array